In [1]:
import requests
import json
import requests
from bs4 import BeautifulSoup
import csv


# Load URLs from JSON file
with open('urls.json', 'r') as f:
    urls = json.load(f)

# open a CSV file for writing
with open('news_articles.csv', mode='w', newline='') as csvfile:
    # define the fieldnames for the CSV writer
    fieldnames = ['Topic', 'Content']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()  # write the header row to the CSV file

    for url in urls:
        url_string = url["url"]
        r = requests.get(url_string)

        if r.status_code == 200:
            soup = BeautifulSoup(r.content, "html.parser")
            article = soup.find("article")
            # Get the text out of the soup and print it
            content = article.get_text()
            # print(content)

            # write the topic and content to the CSV file
            writer.writerow({'Topic': url['title'], 'Content': content})
            print(f'Successfully saved {url["title"]} to CSV')
        else:
            print(f"Error: {r.status_code}")

Successfully saved sports to CSV
Successfully saved sports to CSV
Successfully saved sports to CSV
Successfully saved business to CSV
Successfully saved business to CSV
Successfully saved business to CSV
Successfully saved technology to CSV
Successfully saved technology to CSV
Successfully saved technology to CSV
Successfully saved technology to CSV
Successfully saved sports to CSV
Successfully saved sports to CSV
Successfully saved politics to CSV
Successfully saved politics to CSV
Successfully saved politics to CSV
Successfully saved politics to CSV
Successfully saved health to CSV
Successfully saved health to CSV
Successfully saved health to CSV
Successfully saved health to CSV
Successfully saved climate to CSV
Successfully saved climate to CSV
Successfully saved climate to CSV
Successfully saved sports to CSV
Successfully saved climate to CSV
Successfully saved climate to CSV
Successfully saved climate to CSV
Successfully saved climate to CSV
Successfully saved business to CSV
Succ

In [1]:
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Load the news corpus into a Pandas DataFrame
df = pd.read_csv('news_articles.csv')

# Initialize NLTK components
stop_words = set(stopwords.words('english'))
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

# Function to preprocess text data
def preprocess_text(Content):
    # Remove punctuation
    Content = Content.translate(str.maketrans('', '', string.punctuation))
    
    # Remove digits
    Content = re.sub(r'\d+', '', Content)
    
    # Tokenize the text into words
    words = word_tokenize(Content.lower())

    # Remove stop words
    words = [w for w in words if not w in stop_words]

    # Lemmatize the remaining words
    words = [lemmatizer.lemmatize(w) for w in words]

    # Rejoin the words into a single string
    return ' '.join(words)

# Apply the preprocessing function to the news corpus and save into a new CSV file
df['Content'] = df['Content'].apply(preprocess_text)
df.to_csv('preprocessed_news_corpus.csv', index=False)
print(f'Successfully saved preprocessed data to csv')

# print(df)

[nltk_data] Error loading wordnet: <urlopen error [Errno 104]
[nltk_data]     Connection reset by peer>


         Topic                                            Content
0       sports  manchester united premier league club owe almo...
1       sports  felix jones england recruit exireland fullback...
2       sports  premier league tightens ownership rule stop hu...
3     business  happening house price could crashpublished hou...
4     business  india middle class hit rising cost livingpubli...
5     business  bank england heightened alert banking turmoilp...
6   technology  elon musk among expert urging halt ai training...
7   technology  elon musk twitter bos announces blue tick shak...
8   technology  fun apps banned french official phonespublishe...
9   technology  google face new multibillion advertising lawsu...
10      sports  man city v liverpool danny murphy jurgen klopp...
11      sports  premier league relegation nine team danger dro...
12    politics  donald trump indictment exus president charged...
13    politics  humza yousafs first fmqs hit series disruption...
14    poli

Vectorized using TF-IDF Vectorizer and adjusting the parameters to improve the quality of the features.
Adjusted min_df=4 meaning any word that appears in the corpus less than 4 times will be excluded from the vocabulary.

Training and testing using Naive Bayes model.

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score



# Load the preprocessed news corpus
df = pd.read_csv('preprocessed_news_corpus.csv')

# Create a TF-IDF Vectorizer object
vectorizer = TfidfVectorizer()
# vectorizer = TfidfVectorizer(max_df=0.5)
# vectorizer = TfidfVectorizer(min_df=4)


# Vectorize the text data
X = vectorizer.fit_transform(df['Content'])

# Get the labels
y = df['Topic']

# split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_train, y_train)

# Predict the categories for the test set
y_pred = clf.predict(X_test)

# Evaluate the performance of the classifier
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred, average='weighted', zero_division=1))
print("Recall:", recall_score(y_test, y_pred, average='weighted', zero_division=1))
print("F1-Score:", f1_score(y_test, y_pred, average='weighted'))

Accuracy: 0.14285714285714285
Precision: 1.0
Recall: 0.14285714285714285
F1-Score: 0.21428571428571427


Here, instead of Tf-idfVectorizer, I've used CountVectorizer. 
Training and testing with Random Forest classifier model.

In [15]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the preprocessed news corpus
df = pd.read_csv('preprocessed_news_corpus.csv')

# Create a CountVectorizer object
vectorizer = CountVectorizer()

# Vectorize the text data
X = vectorizer.fit_transform(df['Content'])

# Get the labels
y = df['Topic']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Random Forest classifier
clf = RandomForestClassifier(n_estimators=150, random_state=42)
clf.fit(X_train, y_train)

# Predict the categories for the test set
y_pred = clf.predict(X_test)

# Evaluate the performance of the classifier
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred, average='weighted', zero_division=1))
print("Recall:", recall_score(y_test, y_pred, average='weighted', zero_division=1))
print("F1-Score:", f1_score(y_test, y_pred, average='weighted'))

Accuracy: 0.5714285714285714
Precision: 0.7857142857142857
Recall: 0.5714285714285714
F1-Score: 0.6285714285714287


/home/silver/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/silver/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
